Antes de empezar a trabajar con la BBDD de Northwind, conozcamos un poco las BBDD que tenemos en nuestro servidor. ¿Qué BBDD tenemos en nuestro servidor?

In [1]:
import mysql.connector
import pandas as pd

In [2]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='northwind',
                              auth_plugin='mysql_native_password')


In [3]:
mycursor = cnx.cursor()
mycursor.execute('SHOW DATABASES')
for x in mycursor:
    print(x)

('BD_pruebas',)
('adalab',)
('information_schema',)
('leccion-1-sql',)
('leccion-10-sql',)
('leccion-11-sql',)
('leccion-12-sql',)
('leccion-13-sql',)
('leccion-14-sql',)
('leccion-2-sql',)
('leccion-3-sql',)
('leccion-4-sql',)
('leccion-5-sql',)
('leccion-6-sql',)
('leccion-7-sql',)
('leccion-8-sql',)
('leccion-9-sql',)
('musica_adalabers',)
('mysql',)
('northwind',)
('performance_schema',)
('proyecto',)
('sakila',)
('sports_analytics',)
('sys',)
('tienda',)
('tienda_zapatillas',)
('world',)


Empezemos a explorar la BBDD de Northwind. ¿Qué tablas componen la BBDD? Recuerda que primero nos tendremos que conectar con la BBDD sobre la que queremos trabajar. Guarda el resultado de los nombres de la tablas en una lista.

In [16]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='northwind',
                              auth_plugin='mysql_native_password')

In [19]:
tablas_northwind = []
mycursor = cnx.cursor()
mycursor.execute('SHOW TABLES')
for x in mycursor:
    tablas_northwind.append(x[0])

print(tablas_northwind)

['categories', 'customer_customer_demo', 'customer_demographics', 'customers', 'employee_territories', 'employees', 'order_details', 'orders', 'products', 'region', 'shippers', 'suppliers', 'territories']


¿Qué columnas tiene cada una de las tablas? Para esto tendréis que hacer un for loop para que nos saque el resultado de todas las columnas.

In [37]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='INFORMATION_SCHEMA',
                              auth_plugin='mysql_native_password')

In [38]:
mycursor = cnx.cursor()
for i in tablas_northwind:
    mycursor.execute(f'SELECT COLUMN_NAME FROM COLUMNS WHERE TABLE_NAME = "{i}" AND TABLE_SCHEMA = "northwind"')
    print(f'Estas son las columnas de la tabla {i}:')
    for x in mycursor:
        print(x[0])

Estas son las columnas de la tabla categories:
category_id
category_name
description
picture
Estas son las columnas de la tabla customer_customer_demo:
customer_id
customertype_id
Estas son las columnas de la tabla customer_demographics:
customer_desc
customer_type_id
Estas son las columnas de la tabla customers:
address
city
company_name
contact_name
contact_title
country
customer_id
fax
phone
postal_code
region
Estas son las columnas de la tabla employee_territories:
employee_id
territory_id
Estas son las columnas de la tabla employees:
address
birth_date
city
country
employee_id
extension
first_name
hire_date
home_phone
last_name
notes
photo
photo_path
postal_code
region
reports_to
title
title_of_courtesy
Estas son las columnas de la tabla order_details:
discount
order_id
product_id
quantity
unit_price
Estas son las columnas de la tabla orders:
customer_id
employee_id
freight
order_date
order_id
required_date
ship_address
ship_city
ship_country
ship_name
ship_postal_code
ship_region

Hagamos unas queries facilitas:

Extraed el primer pedido donde se ha gastado más. Devolved el id del pedido y la cantidad gastada.

In [44]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='northwind',
                              auth_plugin='mysql_native_password')

sql = "SELECT order_id, SUM((unit_price * quantity) - (unit_price * quantity * discount)) AS 'Precio' FROM order_details GROUP BY order_id ORDER BY 'Precio'DESC LIMIT 1"

df = pd.read_sql_query(sql, cnx)

df.head()

,order_id,Precio
0,10248,440.0


Devuelve la misma query que en el anterior ejercicio, pero en este caso devuelve todos los resultados.

In [45]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='northwind',
                              auth_plugin='mysql_native_password')

sql = "SELECT order_id, SUM((unit_price * quantity) - (unit_price * quantity * discount)) AS 'Precio' FROM order_details GROUP BY order_id ORDER BY 'Precio'DESC"

df = pd.read_sql_query(sql, cnx)

df

,order_id,Precio
0,10248,440.000000
1,10249,1863.400000
2,10250,1552.599983
3,10251,654.060000
4,10252,3597.900000
...,...,...
825,11073,300.000000
826,11074,232.085000
827,11075,498.099994
828,11076,792.750000


Extraed el números de pedidos que se hayan hecho por dia. Devuelve los resultados en una dataframe.

In [47]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='northwind',
                              auth_plugin='mysql_native_password')

sql = "SELECT COUNT(order_id) AS 'NumeroPedidos', order_date FROM orders GROUP BY order_date"

df = pd.read_sql_query(sql, cnx)

df

,NumeroPedidos,order_date
0,1,1996-07-04
1,1,1996-07-05
2,2,1996-07-08
3,1,1996-07-09
4,1,1996-07-10
...,...,...
475,4,1998-04-30
476,3,1998-05-01
477,3,1998-05-04
478,4,1998-05-05


Extraer los pedidos hechos por "Nancy Davolio". Tendréis que hacer una subquery.

In [51]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='northwind',
                              auth_plugin='mysql_native_password')

sql = "SELECT order_id AS 'NumeroPedidos' FROM orders WHERE employee_id IN (SELECT employee_id FROM employees WHERE first_name = 'Nancy' AND last_name = 'Davolio')"

df = pd.read_sql_query(sql, cnx)

df

,NumeroPedidos
0,10258
1,10270
2,10275
3,10285
4,10292
...,...
118,11064
119,11067
120,11069
121,11071


Para finalizar el ejercicio, desconectad la conexión con el servidor.

In [52]:
cnx.close()